In [5]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
from pprint import pprint
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier # rf분류기

In [10]:
#### load data set
## train
X_train1 = pd.read_csv('temp_data/X_train_stat.csv')
X_train1.columns = ['new_id'] + [x +'_stat' for x in X_train1.columns[1:]]
X_train2 = pd.read_csv('temp_data/X_train_easy_time.csv').drop('new_id',axis=1)
X_train2.columns = [x +'_basic_time' for x in X_train2.columns]
X_train3 = pd.read_csv('temp_data/X_train_게임활동_time.csv').drop('new_id',axis=1)
X_train3.columns = [x +'_time_series' for x in X_train3.columns]
X_train6 = pd.read_csv('temp_data/X_train_ratio.csv').drop('new_id',axis=1)
X_train6.fillna(0.0)
X_train9 = pd.read_csv('temp_data/X_train_act_comb_1.csv').drop('new_id',axis=1)
X_train10 = pd.read_csv('temp_data/train_playpattern_mean_encoding.csv').drop('new_id',axis=1)

## test
X_test1 = pd.read_csv('temp_data/X_test_stat.csv')
X_test1.columns = X_train1.columns
X_test2 = pd.read_csv('temp_data/X_test_easy_time.csv').drop('new_id',axis=1)
X_test2.columns = X_train2.columns
X_test3 = pd.read_csv('temp_data/X_test_게임활동_time.csv').drop('new_id',axis=1)
X_test3.columns = X_train3.columns
X_test6 = pd.read_csv('temp_data/X_test_ratio.csv').drop('new_id',axis=1)
X_test6.fillna(0.0)
X_test9 = pd.read_csv('temp_data/X_test_act_comb_1.csv').drop('new_id',axis=1)
X_test10 = pd.read_csv('temp_data/test_playpattern_mean_encoding.csv').drop('new_id',axis=1)


## guild and trade and party
X_train4 = pd.read_csv('temp_data/temp_guild_train.csv').drop('new_id',axis=1)
X_train5 = pd.read_csv('temp_data/temp_trade_train.csv').drop('new_id',axis=1)
X_test4 = pd.read_csv('temp_data/temp_guild_test.csv').drop('new_id',axis=1)
X_test5 = pd.read_csv('temp_data/temp_trade_test.csv').drop('new_id',axis=1)
X_train7 = pd.read_csv('temp_data/X_train_party.csv').drop('new_id',axis=1)
X_test7 = pd.read_csv('temp_data/X_test_party.csv').drop('new_id',axis=1)
X_train8 = pd.read_csv('temp_data/training_trade.csv').drop('new_id',axis=1)
X_test8 = pd.read_csv('temp_data/test_trade.csv').drop('new_id',axis=1)

In [11]:
X_train1

,new_id,payment_amount_CAT_stat,payment_amount_min_stat,payment_amount_max_stat,payment_amount_range_stat,payment_amount_median_stat,payment_amount_sum_stat,cnt_dt_CAT_stat,cnt_dt_min_stat,cnt_dt_max_stat,...,guild_chat_sum_stat,making_cnt_CAT_stat,making_cnt_min_stat,making_cnt_max_stat,making_cnt_range_stat,making_cnt_median_stat,making_cnt_sum_stat,cnt_enter_bam_CAT_stat,faction_chat_CAT_stat,gathering_cnt_CAT_stat
0,tr0,0,NaN,NaN,NaN,NaN,NaN,1,1.0,1.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
1,tr1,0,NaN,NaN,NaN,NaN,NaN,1,1.0,1.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
2,tr2,0,NaN,NaN,NaN,NaN,NaN,1,2.0,7.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
3,tr3,0,NaN,NaN,NaN,NaN,NaN,1,2.0,2.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
4,tr4,1,3960.0,178200.0,174240.0,69300.0,407010.0,1,6.0,7.0,...,10501.0,1,2.0,35.0,33.0,10.5,78.0,0,0,0
5,tr5,0,NaN,NaN,NaN,NaN,NaN,1,3.0,6.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
6,tr6,0,NaN,NaN,NaN,NaN,NaN,1,5.0,5.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
7,tr7,0,NaN,NaN,NaN,NaN,NaN,1,2.0,4.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
8,tr8,0,NaN,NaN,NaN,NaN,NaN,1,2.0,2.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0
9,tr9,0,NaN,NaN,NaN,NaN,NaN,1,4.0,5.0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,0,0


In [3]:
X_train = pd.concat((X_train1,X_train2,X_train3,X_train4,X_train5, X_train6,X_train7,X_train8,X_train9,X_train10),axis=1).drop('new_id',axis=1)
X_test = pd.concat((X_test1,X_test2,X_test3,X_test4,X_test5,X_test6,X_test7,X_test8,X_test9,X_test10),axis=1).drop('new_id',axis=1)

---

In [4]:
corr_set = X_train.fillna(0.0).corr()

In [5]:
uppers = np.triu(np.ones((len(corr_set),len(corr_set)),dtype=np.bool)) & ~np.identity(len(corr_set),dtype=np.bool)
df_cor = corr_set.where(uppers).stack().reset_index()
df_cor.columns=['feature_1','feature_2','correlation']

In [6]:
temp = df_cor.sort_values(by='correlation',ascending=False)

In [7]:
temp

,feature_1,feature_2,correlation
35081,cnt_enter_inzone_light_max_stat,cnt_enter_inzone_normal_max_stat,0.992189
10740,get_money_max_stat,get_money_range_stat,0.989983
28934,duel_cnt_median_stat,duel_cnt_sum_stat,0.989179
28150,duel_cnt_max_stat,duel_cnt_median_stat,0.988736
68442,guild_chat_MA_1_basic_time,guild_chat_MA_2_basic_time,0.985891
36203,cnt_enter_inzone_light_sum_stat,cnt_enter_inzone_normal_sum_stat,0.985813
77613,inzone_light_median_basic_time,inzone_normal_median_basic_time,0.985183
45885,normal_chat_max_stat,normal_chat_range_stat,0.984729
52619,party_chat_sum_stat,party_chat_median_basic_time,0.984304
69255,guild_chat_MA_4_basic_time,guild_chat_MA_5_basic_time,0.984032
